In [17]:
!pip install qiskit qiskit-aer

In [18]:
# Quantum Phase Estimation (QPE) using Qiskit 2.x

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

def qpe_circuit(num_count_qubits, unitary, theta):
    """
    Constructs the Quantum Phase Estimation (QPE) circuit.
    num_count_qubits: number of counting qubits
    unitary: unitary gate (to apply controlled operations)
    theta: phase parameter (for U gate)
    """
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Apply Hadamard gates on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate (|1>) for the target qubit
    qc.x(num_count_qubits)

    # Step 3: Apply controlled unitary operations
    for qubit in range(num_count_qubits):
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    # Step 4: Apply inverse QFT to counting qubits
    inverse_qft(qc, num_count_qubits)

    # Step 5: Measure counting qubits
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    return qc

In [19]:
def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
        qc.h(j)
    return qc

In [20]:
def run_qpe(num_count_qubits=3, theta=0.125):
    """Executes the QPE circuit and visualizes the phase estimation result."""
    simulator = AerSimulator()
    qc = qpe_circuit(num_count_qubits, "U", theta)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=2048).result()
    counts = result.get_counts()
    plot_histogram(counts)
    plt.show()
    print(qc.draw(output='text'))

In [21]:
if __name__ == "__main__":
    num_count_qubits = 3
    theta = 0.125  # phase value (1/8)
    print(f"Running Quantum Phase Estimation with {num_count_qubits} counting qubits and phase {theta}")
    run_qpe(num_count_qubits, theta)

Running Quantum Phase Estimation with 3 counting qubits and phase 0.125
     ┌───┐                            ┌───┐                                   »
q_0: ┤ H ├─■────────────────────────X─┤ H ├─■──────────────■──────────────────»
     ├───┤ │                        │ └───┘ │P(-π/2) ┌───┐ │                  »
q_1: ┤ H ├─┼────────■───────────────┼───────■────────┤ H ├─┼─────────■────────»
     ├───┤ │        │               │                └───┘ │P(-π/4)  │P(-π/2) »
q_2: ┤ H ├─┼────────┼────────■──────X──────────────────────■─────────■────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)                                            »
q_3: ┤ X ├─■────────■────────■────────────────────────────────────────────────»
     └───┘                                                                    »
c: 3/═════════════════════════════════════════════════════════════════════════»
                                                                              »
«     ┌─┐           
«q_0: ┤M├───────────
«     

#Change the Phase Value Try different values of theta (e.g., 0.25, 0.375, 0.5) and see how the measured output changes.

In [22]:
# --- Imports ---
!pip install qiskit qiskit-aer
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error

print("Libraries imported and helper functions defined.")

# --- Core QPE Circuit (from your script) ---
def qpe_circuit(num_count_qubits, theta):
    """
    Constructs the Quantum Phase Estimation (QPE) circuit.
    """
    qc = QuantumCircuit(num_count_qubits + 1, num_count_qubits)

    # Step 1: Apply Hadamard gates on counting qubits
    qc.h(range(num_count_qubits))

    # Step 2: Prepare eigenstate (|1>) for the target qubit
    qc.x(num_count_qubits)

    # Step 3: Apply controlled unitary operations
    for qubit in range(num_count_qubits):
        # qc.cp(lambda, control, target)
        qc.cp(2 * np.pi * theta * (2 ** qubit), qubit, num_count_qubits)

    return qc

# --- Inverse QFT (from your script) ---
def inverse_qft(qc, n):
    """Apply the inverse Quantum Fourier Transform on n qubits."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n):
        for k in range(j):
            qc.cp(-np.pi / 2 ** (j - k), k, j)
        qc.h(j)
    return qc

# --- Helper Function for Running Simulation ---
def run_simulation(qc, noise_model=None, task_name=""):
    """Helper to run simulation and plot results."""
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)

    job_kwargs = {"shots": 2048}
    if noise_model:
        job_kwargs["noise_model"] = noise_model
        print(f"\n--- Running {task_name} with Noise ---")
    else:
        print(f"\n--- Running {task_name} (Ideal Simulation) ---")

    result = simulator.run(compiled_circuit, **job_kwargs).result()
    counts = result.get_counts()

    print(f"Result counts: {counts}")

    # Plot histogram
    plot_title = f"{task_name} Results"
    if noise_model:
        plot_title += " (Noisy)"

    plot_histogram(counts, title=plot_title)
    plt.show()

Libraries imported and helper functions defined.


In [23]:
# --- Task 1: Change the Phase Value ---

def run_task_1_different_phases():
    """
    Runs QPE for theta = 0.25, 0.375, and 0.5 with 3 counting qubits.
    """
    num_count_qubits = 3
    phases = [0.25, 0.375, 0.5]

    print("### Running Task 1: Different Phase Values ###")
    print(f"Using {num_count_qubits} counting qubits.")

    for theta in phases:
        # Expected value calculation
        phase_val = int(round(theta * (2**num_count_qubits)))
        expected_bin = format(phase_val, f'0{num_count_qubits}b')

        task_name = f"QPE for theta = {theta} (1 / {1/theta:.0f})"
        print(f"\nRunning for theta = {theta}. Expected register: {phase_val} -> '{expected_bin}'")

        qc = qpe_circuit(num_count_qubits, theta)
        inverse_qft(qc, num_count_qubits)
        qc.measure(range(num_count_qubits), range(num_count_qubits))

        run_simulation(qc, task_name=task_name)

    print("\nObservation for Task 1:")
    print("The measured bitstring corresponds to the numerator of the phase.")
    print("theta=0.25 (1/4) -> 0.25 * 8 = 2 -> '010'")
    print("theta=0.375 (3/8) -> 0.375 * 8 = 3 -> '011'")
    print("theta=0.5 (1/2 = 4/8) -> 0.5 * 8 = 4 -> '100'")

# --- Run Task 1 ---
run_task_1_different_phases()

### Running Task 1: Different Phase Values ###
Using 3 counting qubits.

Running for theta = 0.25. Expected register: 2 -> '010'

--- Running QPE for theta = 0.25 (1 / 4) (Ideal Simulation) ---
Result counts: {'010': 2048}

Running for theta = 0.375. Expected register: 3 -> '011'

--- Running QPE for theta = 0.375 (1 / 3) (Ideal Simulation) ---
Result counts: {'011': 2048}

Running for theta = 0.5. Expected register: 4 -> '100'

--- Running QPE for theta = 0.5 (1 / 2) (Ideal Simulation) ---
Result counts: {'100': 2048}

Observation for Task 1:
The measured bitstring corresponds to the numerator of the phase.
theta=0.25 (1/4) -> 0.25 * 8 = 2 -> '010'
theta=0.375 (3/8) -> 0.375 * 8 = 3 -> '011'
theta=0.5 (1/2 = 4/8) -> 0.5 * 8 = 4 -> '100'


#Increase the Number of Counting Qubits Use 4 or 5 counting qubits for higher precision phase estimation.

In [24]:
# --- Task 2: Increase the Number of Counting Qubits ---

def run_task_2_higher_precision():
    """
    Compares 3-qubit vs 5-qubit QPE to show precision.
    Uses a phase (1/3) that is not perfectly representable.
    """
    theta = 1/3  # ~0.333...

    print("### Running Task 2: Higher Precision ###")
    print(f"Estimating phase theta = 1/3 ({theta:.5f})...")
    print("This phase cannot be perfectly represented in binary.")

    # --- 3 Qubits ---
    n_count_3 = 3
    qc3 = qpe_circuit(n_count_3, theta)
    inverse_qft(qc3, n_count_3)
    qc3.measure(range(n_count_3), range(n_count_3))

    phase_val_3 = theta * (2**n_count_3)
    print(f"\n3 Qubits: 2^3 * (1/3) = {phase_val_3:.3f}. Expect results near 2 ('010') and 3 ('011').")
    run_simulation(qc3, task_name="3-Qubit QPE for theta=1/3")

    # --- 5 Qubits ---
    n_count_5 = 5
    qc5 = qpe_circuit(n_count_5, theta)
    inverse_qft(qc5, n_count_5)
    qc5.measure(range(n_count_5), range(n_count_5))

    phase_val_5 = theta * (2**n_count_5)
    print(f"\n5 Qubits: 2^5 * (1/3) = {phase_val_5:.3f}. Expect results near 10 ('01010') and 11 ('01011').")
    run_simulation(qc5, task_name="5-Qubit QPE for theta=1/3")

    print("\nObservation for Task 2:")
    print("3 qubits estimates 1/3 as ~2/8 (0.25) or ~3/8 (0.375).")
    print("5 qubits estimates 1/3 as ~10/32 (0.3125) or ~11/32 (0.34375).")
    print("The 5-qubit estimate is much closer to the true value of 0.333...!")

# --- Run Task 2 ---
run_task_2_higher_precision()

### Running Task 2: Higher Precision ###
Estimating phase theta = 1/3 (0.33333)...
This phase cannot be perfectly represented in binary.

3 Qubits: 2^3 * (1/3) = 2.667. Expect results near 2 ('010') and 3 ('011').

--- Running 3-Qubit QPE for theta=1/3 (Ideal Simulation) ---
Result counts: {'110': 24, '101': 35, '000': 33, '010': 368, '001': 63, '100': 99, '111': 27, '011': 1399}

5 Qubits: 2^5 * (1/3) = 10.667. Expect results near 10 ('01010') and 11 ('01011').

--- Running 5-Qubit QPE for theta=1/3 (Ideal Simulation) ---
Result counts: {'00000': 1, '11101': 1, '10011': 2, '01110': 18, '10100': 4, '11011': 3, '10000': 7, '10010': 4, '00001': 1, '10111': 6, '10101': 2, '01011': 1447, '00011': 4, '00111': 13, '01010': 324, '00100': 6, '01100': 80, '00110': 6, '01000': 29, '01101': 21, '10110': 3, '01111': 4, '11111': 4, '11000': 3, '10001': 5, '11010': 1, '01001': 45, '00101': 4}

Observation for Task 2:
3 qubits estimates 1/3 as ~2/8 (0.25) or ~3/8 (0.375).
5 qubits estimates 1/3 as ~1

#Compare with Theoretical Output Calculate the expected binary representation of the phase and compare with simulation results.

In [25]:
# --- Task 3: Compare with Theoretical Output ---

def run_task_3_theoretical_comparison():
    """
    Runs QPE and explicitly prints the theoretical vs. simulated result.
    """
    num_count_qubits = 4
    theta = 0.625  # 5/8

    print("### Running Task 3: Theoretical Comparison ###")
    print(f"Using {num_count_qubits} counting qubits and theta = {theta}.")

    # --- Theoretical Calculation ---
    phase_register_value = theta * (2**num_count_qubits)
    phase_register_int = int(round(phase_register_value))
    expected_binary = format(phase_register_int, f'0{num_count_qubits}b')

    print(f"\n--- Theoretical Expectation ---")
    print(f"Calculation: 2^{num_count_qubits} * {theta} = {phase_register_value}")
    print(f"Expected integer result: {phase_register_int}")
    print(f"Expected binary string (measurement): '{expected_binary}'")

    # --- Simulation ---
    qc = qpe_circuit(num_count_qubits, theta)
    inverse_qft(qc, num_count_qubits)
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    run_simulation(qc, task_name=f"QPE for theta={theta}")

    print("\nObservation for Task 3:")
    print("The ideal simulation result '1010' matches the theoretical expectation exactly.")

# --- Run Task 3 ---
run_task_3_theoretical_comparison()

### Running Task 3: Theoretical Comparison ###
Using 4 counting qubits and theta = 0.625.

--- Theoretical Expectation ---
Calculation: 2^4 * 0.625 = 10.0
Expected integer result: 10
Expected binary string (measurement): '1010'

--- Running QPE for theta=0.625 (Ideal Simulation) ---
Result counts: {'1010': 2048}

Observation for Task 3:
The ideal simulation result '1010' matches the theoretical expectation exactly.


#Inverse QFT Visualization Add a qc.draw('mpl') command before measurement to visualize the inverse QFT structure.

In [27]:
import sys
# Tactic 1: Force-install into the kernel's specific environment
!{sys.executable} -m pip install pylatexenc

# --- Task 4: Inverse QFT Visualization ---

def run_task_4_visualize_iqft():
    """
    Draws the circuit with qc.draw('mpl') before adding measurements.
    """
    num_count_qubits = 3
    theta = 0.125  # 1/8

    print("### Running Task 4: iQFT Visualization ###")
    print("Building circuit...")

    qc = qpe_circuit(num_count_qubits, theta)
    qc.barrier()
    inverse_qft(qc, num_count_qubits)
    qc.barrier()

    print("Displaying circuit diagram (plt.show())...")

    # Tactic 2: Tell the drawer NOT to use LaTeX, bypassing pylatexenc
    try:
        circuit_diagram = qc.draw('mpl', style={'usetex': False})
    except Exception as e:
        print(f"Drawing failed with 'usetex=False'. Error: {e}")
        print("Falling back to text-based drawing:")
        print(qc.draw('text'))
        return # Exit the function if drawing fails

    plt.show()

    # Now, add measurements to run the simulation
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    print("--- Circuit Diagram Shown ---")
    print("Now, running the simulation...")
    run_simulation(qc, task_name=f"QPE for theta={theta} (post-visualization)")

    print("\nObservation for Task 4:")
    print("A plot of the circuit diagram was displayed before the simulation was run.")

# --- Run Task 4 ---
# NOTE: After the pip install, you *might* need to restart the kernel
# one last time. But try running this cell directly first. It might just work.
run_task_4_visualize_iqft()

### Running Task 4: iQFT Visualization ###
Building circuit...
Displaying circuit diagram (plt.show())...
Drawing failed with 'usetex=False'. Error: "The 'pylatexenc' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install pylatexenc'."
Falling back to text-based drawing:
     ┌───┐                          ░    ┌───┐                         »
q_0: ┤ H ├─■────────────────────────░──X─┤ H ├─■──────────────■────────»
     ├───┤ │                        ░  │ └───┘ │P(-π/2) ┌───┐ │        »
q_1: ┤ H ├─┼────────■───────────────░──┼───────■────────┤ H ├─┼────────»
     ├───┤ │        │               ░  │                └───┘ │P(-π/4) »
q_2: ┤ H ├─┼────────┼────────■──────░──X──────────────────────■────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)  ░                                  »
q_3: ┤ X ├─■────────■────────■──────░──────────────────────────────────»
     └───┘                          ░                                  »
c: 3/═════════════════════════════════════

#Noise Simulation Introduce a NoiseModel using Qiskit Aer and observe how it affects accuracy.

In [28]:
# --- Task 5: Noise Simulation ---

def run_task_5_noise_simulation():
    """
    Introduces a noise model and observes the effect on accuracy.
    """
    num_count_qubits = 3
    theta = 0.125  # 1/8

    print("### Running Task 5: Noise Simulation ###")
    print(f"Using {num_count_qubits} qubits, theta = {theta}. Ideal result is '001'.")

    # --- 1. Create a Noise Model ---
    noise_model = NoiseModel()

    p_error_1 = 0.005  # 0.5% error on single-qubit gates
    p_error_2 = 0.05   # 5% error on two-qubit gates (like cp)

    error_1 = depolarizing_error(p_error_1, 1)
    error_2 = depolarizing_error(p_error_2, 2)

    noise_model.add_all_qubit_quantum_error(error_1, ['h', 'x'])
    noise_model.add_all_qubit_quantum_error(error_2, ['cp', 'swap'])

    print(f"\nCreated a noise model with:")
    print(f"  {p_error_1*100}% depolarizing error on 'h' and 'x' gates.")
    print(f"  {p_error_2*100}% depolarizing error on 'cp' and 'swap' gates.")

    # --- 2. Build the Circuit ---
    qc = qpe_circuit(num_count_qubits, theta)
    inverse_qft(qc, num_count_qubits)
    qc.measure(range(num_count_qubits), range(num_count_qubits))

    # --- 3. Run Simulation with Noise ---
    run_simulation(qc, noise_model=noise_model, task_name=f"Noisy QPE for theta={theta}")

    print("\nObservation for Task 5:")
    print("The ideal result '001' is still the most probable, but the noise")
    print("introduces other incorrect results ('000', '011', etc.) with significant probability.")

# --- Run Task 5 ---
run_task_5_noise_simulation()

### Running Task 5: Noise Simulation ###
Using 3 qubits, theta = 0.125. Ideal result is '001'.

Created a noise model with:
  0.5% depolarizing error on 'h' and 'x' gates.
  5.0% depolarizing error on 'cp' and 'swap' gates.

--- Running Noisy QPE for theta=0.125 with Noise ---
Result counts: {'100': 56, '110': 11, '010': 62, '000': 116, '101': 67, '111': 50, '011': 54, '001': 1632}

Observation for Task 5:
The ideal result '001' is still the most probable, but the noise
introduces other incorrect results ('000', '011', etc.) with significant probability.
